In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install efficientnet_pytorch torchvision

In [ ]:
import torch
from efficientnet_pytorch import EfficientNet
import torch.nn as nn

# Load the EfficientNet-B4 model architecture
model = EfficientNet.from_pretrained('efficientnet-b2')
model._fc = nn.Linear(model._fc.in_features, 1)  # modify final layer

# Load your trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/eye_dataset/trained_model/trained_model_1.pth', map_location=torch.device('cpu')))
model.eval()  # evaluation mode


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth
100%|██████████| 35.1M/35.1M [00:00<00:00, 106MB/s]


Loaded pretrained weights for efficientnet-b2


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
from PIL import Image
from torchvision import transforms
import torch

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((260, 260)),  # EfficientNet-B2 expects 260x260 input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet normalization
                         [0.229, 0.224, 0.225]),
])

# Load and preprocess the image
image = Image.open('/content/drive/MyDrive/eye_dataset/Test_Set/Test/245.png').convert('RGB')
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension
input_tensor = input_tensor.to(device)  # device = 'cuda' or 'cpu'

# Inference
model.eval()  # Make sure model is in evaluation mode
with torch.no_grad():
    output = model(input_tensor)
    prediction = torch.sigmoid(output)  # because final output is 1 value (binary)

# Interpret prediction
pred_class = (prediction > 0.5).int()  # threshold at 0.5

print('Prediction score:', prediction.item())
print('Predicted class:', pred_class.item())


Prediction score: 0.33755555748939514
Predicted class: 0


In [ ]:
import pandas as pd

# Load the CSV file
test_file = pd.read_csv("/content/drive/MyDrive/eye_dataset/Test_Set/normalities_prob.csv")

# Function to get disease risk by ID
def get_disease_risk_by_id(id_value):
    result = test_file.loc[test_file['ID'] == id_value, 'Disease_Risk']
    if not result.empty:
        return result.values[0]
    else:
        return None  # or you can raise an error if you prefer

# Example usage
print(get_disease_risk_by_id(245))  # should print 1


0
